# 04 MS Semantic Kernel - Memory

## Intro 

Semantic Kernel simplifies storing and querying information by abstracting necessary steps like, e.g. creating embeddings, storing embeddings etc.

## Step 1 - Read Environment / Create Semantic Kernel Instance

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.0.0-beta1"
#r "nuget: DotNetEnv, 2.5.0"

using Azure; 
using Azure.AI.OpenAI;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Plugins.Memory;

using DotNetEnv;

string _configurationFile = @"../01/application.env";
Env.Load(_configurationFile);

string oAiApiKey = Environment.GetEnvironmentVariable("AOAI_APIKEY");
string oAiEndpoint = Environment.GetEnvironmentVariable("AOAI_ENDPOINT");
string chatCompletionDeploymentName = Environment.GetEnvironmentVariable("CHATCOMPLETION_DEPLOYMENTNAME");
string embeddingDeploymentName = Environment.GetEnvironmentVariable("EMBEDDING_DEPLOYMENTNAME");

IKernel kernel = new KernelBuilder()
    .WithAzureChatCompletionService(
        endpoint: oAiEndpoint, 
        apiKey: oAiApiKey,
        deploymentName: chatCompletionDeploymentName)
    .Build();

Console.WriteLine($"Semantic Kernel created...");


Installed Packages DotNetEnv, 2.5.0 Microsoft.SemanticKernel, 1.0.0-beta1

Semantic Kernel created...


## Step 2 - Download and Summarize

In [2]:
using System.IO;

#!import ./SK_Memory_DownloadSummarize.cs

Dictionary<string,string> summarizations = await DownloadSummarize.Execute(kernel);

foreach (KeyValuePair<string, string> summarization in summarizations) {
    File.WriteAllText(
        path: $"./Summarizations/{summarization.Key.Split(";")[1].Trim()}.txt", 
        contents: summarization.Value
    );
}


Content downloaded from https://en.wikipedia.org/w/api.php?action=query&format=json&prop=extracts&titles=Champions%20League&explaintext=1&exsectionformat=plain...
Summarized content from https://en.wikipedia.org/w/api.php?action=query&format=json&prop=extracts&titles=Champions%20League&explaintext=1&exsectionformat=plain...
Content downloaded from https://en.wikipedia.org/w/api.php?action=query&format=json&prop=extracts&titles=Super%20Bowl&explaintext=1&exsectionformat=plain...
Summarized content from https://en.wikipedia.org/w/api.php?action=query&format=json&prop=extracts&titles=Super%20Bowl&explaintext=1&exsectionformat=plain...
Content downloaded from https://en.wikipedia.org/w/api.php?action=query&format=json&prop=extracts&titles=Cricket%20World%20Cup&explaintext=1&exsectionformat=plain...
Summarized content from https://en.wikipedia.org/w/api.php?action=query&format=json&prop=extracts&titles=Cricket%20World%20Cup&explaintext=1&exsectionformat=plain...


## Step 3 - Create Semantic Kernel Memory Builder

In [3]:
using Microsoft.SemanticKernel.Memory; 

ISemanticTextMemory memoryBuilder = new MemoryBuilder()
    .WithAzureTextEmbeddingGenerationService(
        endpoint: oAiEndpoint, 
        apiKey: oAiApiKey,
        deploymentName: embeddingDeploymentName)
    .WithMemoryStore(new VolatileMemoryStore())
    .Build();

Console.WriteLine($"Memory Builder created...");

Memory Builder created...


## Step 4 - Store summarizations

In [4]:
string memoryCollectionName = "SportEvents";

foreach (var summarization in summarizations) {

    await memoryBuilder.SaveReferenceAsync(
        collection: memoryCollectionName,
        description: summarization.Key.Split(";")[1].Trim(),
        text: summarization.Value,
        externalId: summarization.Key.Split(";")[2].Trim(),
        externalSourceName: "Wikipedia"
    );
    
    Console.WriteLine($"Summarizations reference saved: '{summarization.Key.Split(";")[1].Trim()}' ...");
}

Summarizations reference saved: 'Champions League' ...
Summarizations reference saved: 'Super Bowl' ...
Summarizations reference saved: 'Cricket World Cup' ...


## Step 5 - Query Memory

In [5]:

string[] queryPhrases = {
    "Europe", 
    "Huge TV audience",
    "Bat-and-ball"
};

foreach (string query in queryPhrases) {
    
    IAsyncEnumerable<MemoryQueryResult> memoryQueryResults = memoryBuilder.SearchAsync(
                collection: memoryCollectionName, 
                query: query, 
                limit:2, 
                minRelevanceScore: 0.75);
    
    await foreach (MemoryQueryResult memoryQueryResult in memoryQueryResults)
    {
        Console.WriteLine($"Query: {query}");
        Console.WriteLine($"  Id:         : {memoryQueryResult.Metadata.Id}");
        Console.WriteLine($"  Description : {memoryQueryResult.Metadata.Description}");
        Console.WriteLine($"  Relevance   : {memoryQueryResult.Relevance}");
    }
}



Query: Europe
  Id:         : https://en.wikipedia.org/w/api.php?action=query&format=json&prop=extracts&titles=Champions%20League&explaintext=1&exsectionformat=plain
  Description : Champions League
  Relevance   : 0.7627838573303226
Query: Huge TV audience
  Id:         : https://en.wikipedia.org/w/api.php?action=query&format=json&prop=extracts&titles=Super%20Bowl&explaintext=1&exsectionformat=plain
  Description : Super Bowl
  Relevance   : 0.7579632475483531
Query: Bat-and-ball
  Id:         : https://en.wikipedia.org/w/api.php?action=query&format=json&prop=extracts&titles=Cricket%20World%20Cup&explaintext=1&exsectionformat=plain
  Description : Cricket World Cup
  Relevance   : 0.778306183402297
Query: Bat-and-ball
  Id:         : https://en.wikipedia.org/w/api.php?action=query&format=json&prop=extracts&titles=Super%20Bowl&explaintext=1&exsectionformat=plain
  Description : Super Bowl
  Relevance   : 0.7573848790959947
